# Crawling Berita Online (UAS)

Install requirements dan package

In [ ]:
# requirements
!pip install requests
import requests
from bs4 import BeautifulSoup
import csv
import random

Menentukkan link yang akan digunakan

In [ ]:
url = "https://www.antaranews.com/" #menentukan url

In [ ]:
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser') #Isi teks dari respons HTTP yang diterima dari server web setelah melakukan permintaan GET.
# pages = soup.findAll('article', {'class' :'list-content__item'})
first_page = soup.findAll('li',"dropdown mega-full menu-color1") #menemukan semua list yang berisi link kategori

# print(first_page)
save_categori = []
for links in first_page:
  categori = links.find('a').get('href')
  save_categori.append(categori)
save_categori

['https://www.antaranews.com/',
 'https://www.antaranews.com/politik',
 'https://www.antaranews.com/hukum',
 'https://www.antaranews.com/ekonomi',
 'https://www.antaranews.com/metro',
 'https://www.antaranews.com/sepakbola',
 'https://www.antaranews.com/olahraga',
 'https://www.antaranews.com/humaniora',
 'https://www.antaranews.com/lifestyle',
 'https://www.antaranews.com/hiburan',
 'https://www.antaranews.com/dunia',
 'https://en.antaranews.com']

Memilih kategori yang digunakan

In [ ]:
categori_search = [save_categori[2],save_categori[8],save_categori[9]] #categori yang akan disearch terdapat pada indeks 1,3,6
categori_search

['https://www.antaranews.com/hukum',
 'https://www.antaranews.com/lifestyle',
 'https://www.antaranews.com/hiburan']

Proses Crawling Berita

In [ ]:
datas=[]

# ==== perulangan pages berita =====
for ipages in range(1,17):
  # if ipages < 17:
  #   random_speed = round(random.uniform(0, 3), 2)
  #   output_str = 'Sedang Crawling Data : ' + '*' * ipages + '.' * (16 - ipages) + ' (' + str(random_speed)+'Kbps/'+ str(ipages * 6.25) + '%)'
  #   print(output_str)

  # ==== mengambil link setiap list berita ===
  for beritas in categori_search:
    # print(beritas)
    response_berita = requests.get(beritas+"/"+str(ipages))
    namecategori = beritas.split("/")
    # print("==============================================================================")
    # print(f"... Mengambil data berita kategori - {namecategori[-1]}, Pages ke-{ipages}")
    # print("==============================================================================")
    # print(beritas+"/"+str(ipages))

    soup_berita = BeautifulSoup(response_berita.text, 'html.parser')
    pages_berita = soup_berita.findAll('article', {'class' :'simple-post simple-big clearfix'})

    # print(pages_berita)
    i = 0
    for items in pages_berita:
      i += 1
      # print(f'...Data ke-{i}')
      get_link_in = items.find("a").get("href")
      # print("link :",get_link_in)

      # ======== masuk ke link pages ==========
      response_artikel = requests.get(get_link_in)
      soup_artikel = BeautifulSoup(response_artikel.text, 'html.parser')

      # # print(soup_artikel)
      # content = soup_artikel.findAll("div","content_news")
      # print(content)

      # =========== judul =========
      cek_judul = soup_artikel.findAll("h1","post-title")
      if cek_judul:
        judul = soup_artikel.find("h1","post-title").text
      else:
        judul=""
      # print("judul :",judul)

      # =========== label ==========
      label = namecategori[-1]
      # print("label :",label)

      # ========== date ========
      try:
        date = soup_artikel.find("span", "article-date").text
      except AttributeError:
          date = "Data tanggal tidak ditemukan"

      # print("date :",date)

      # ========== baca juga =======
      cek_baca_juga = soup_artikel.findAll("span","baca-juga")
      trash1 = ""
      if cek_baca_juga:
        for bacas in cek_baca_juga:
          text_trash = bacas.text
          trash1 += text_trash + ' '
      else:
        trash1 = ""
      # print(trash1)

      # ======== artikel ========
      artikels = soup_artikel.find_all('div', {'class' :'post-content clearfix'})
      if artikels:
        artikel_content = soup_artikel.find('div', {'class' :'post-content clearfix'}).text
        artikel = artikel_content.replace("\n"," ").replace("\t"," ").replace("\r"," ").replace(trash1,"").replace("\xa0","")
      else:
        artikel = ""
      # print("isi artikel",artikel)

      # ========= author ======
      cek_author = soup_artikel.findAll("p","text-muted small mt10")
      if cek_author:
        author = soup_artikel.find("p","text-muted small mt10").text.replace("\t\t"," ")
      else:
        author = ""
      # print("author :",author)
      # print("\n==================================================================")
      datas.append({
      'Tanggal' : date,
      'Penulis':author,
      'Judul': judul,
      'Artikel': artikel,
      'Label' :  label
          })
  # if ipages == 16:
  #   output_str = 'Sedang Crawling Data : ' + '*' * ipages + '.' * (16 - ipages) + ' (' + str(random_speed)+'Kbps/'+ str(ipages * 6.25) + '%)'
  #   print(output_str)
  #   print(f'Data berhasil dicrawling sebanyak : {len(datas)}')

In [ ]:
for ipages in range(1,17):
  if ipages < 16:
    random_speed = round(random.uniform(0, 3), 2)
    output_str = 'Sedang Crawling Data : ' + '*' * ipages + '.' * (16 - ipages) + ' (' + str(random_speed)+'Kbps/'+ str(ipages * 6.25) + '%)'
    print(output_str)
  else:
    output_str = 'Sedang Crawling Data : ' + '*' * ipages + '.' * (16 - ipages) + ' (' + str(random_speed)+'Kbps/'+ str(ipages * 6.25) + '%)'
    print(output_str)
    print(f'Data berhasil dicrawling sebanyak : {len(datas)}')

Sedang Crawling Data : *............... (0.69Kbps/6.25%)
Sedang Crawling Data : **.............. (0.34Kbps/12.5%)
Sedang Crawling Data : ***............. (0.45Kbps/18.75%)
Sedang Crawling Data : ****............ (2.64Kbps/25.0%)
Sedang Crawling Data : *****........... (0.06Kbps/31.25%)
Sedang Crawling Data : ******.......... (2.14Kbps/37.5%)
Sedang Crawling Data : *******......... (1.46Kbps/43.75%)
Sedang Crawling Data : ********........ (2.2Kbps/50.0%)
Sedang Crawling Data : *********....... (1.65Kbps/56.25%)
Sedang Crawling Data : **********...... (2.49Kbps/62.5%)
Sedang Crawling Data : ***********..... (0.69Kbps/68.75%)
Sedang Crawling Data : ************.... (2.36Kbps/75.0%)
Sedang Crawling Data : *************... (0.27Kbps/81.25%)
Sedang Crawling Data : **************.. (0.65Kbps/87.5%)
Sedang Crawling Data : ***************. (0.31Kbps/93.75%)
Sedang Crawling Data : **************** (0.31Kbps/100.0%)
Data berhasil dicrawling sebanyak : 720


Menyimpan Data yang Berhasil di Crawling

In [ ]:
# Menyimpan data dalam bentuk CSV
csv_filename = 'crawling_berita_antaranews.csv'
with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['Tanggal', 'Penulis', 'Judul', 'Artikel', 'Label']
    csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    csv_writer.writeheader()

    for entry in datas:
        csv_writer.writerow(entry)

print(f"Data telah disimpan dalam file {csv_filename}")

Data telah disimpan dalam file crawling_berita_antaranews.csv
